# 0. Introduction

In this notebook the final model for Day-Ahead electricity price forecasting is developed. Among raw data, this model feeds on precomputed bottleneck features. These bottleneck features are
* Hourly price forecasts from a pure time series based model
* Hourly wind and solar power generation for Germany from a renewable generation prediction model


In [1]:
!git clone https://github.com/farwacheema/DA-electricity-price-forecasting
%cd DA-electricity-price-forecasting

Cloning into 'DA-electricity-price-forecasting'...
remote: Enumerating objects: 328, done.
remote: Total 328 (delta 0), reused 0 (delta 0), pack-reused 328
Receiving objects: 100% (328/328), 61.38 MiB | 28.58 MiB/s, done.
Resolving deltas: 100% (144/144), done.
/content/DA-electricity-price-forecasting


In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from keras.models import load_model
from sklearn.externals import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
from sklearn.linear_model import Lasso, LassoLars, LinearRegression, ElasticNet, Ridge, PassiveAggressiveRegressor, \
SGDRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV 
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# 1. Input data


First, we load the historical price dataset, the variable we want to predict with the final model. Additional data in the form of hourly generation per production, hourly forecasted and actual load and renewable generation forcasts are also available. True generation levels as well as forecasts are available on the transparency platform of the German Transmission System Operators. The final model will rely only on load forecasts and bottleneck features from previous models, since these inputs are the only ones available in time for the final prediction in a real world setting. The forecasts and true generation levels are not published prior to market closing and therefore irrlevant for price predictions The other datasets are only considered to give an indication about the model performance under _perfect conditions_. By including this data we are able to identify the margin for improvement of a model that is based on imperfect information. 

In [3]:
# load historical Day-Ahead prices
price_data = pd.read_csv('./raw_data/EPEX_spot_DA_auction_hour_prices_20070720-20170831.csv', parse_dates=True,
                        index_col=0)

In [4]:
# load generation forecasts, actual generation per production type, forecasted and actual load in Germany
actual = pd.read_csv('./processed_data/20150101-20170830-gen_per_prod_type.csv', parse_dates=True, index_col=0)
forecast = pd.read_csv('./processed_data/20150101-20170830-forecast_load_renewable_gen.csv', parse_dates=True, index_col=0)

In [5]:
# sum_forecasts contains solar forecasts and wind power forecasts which are available individually. We can drop
# this dataset
forecast.drop('sum_forecast', axis=1, inplace=True)

In [6]:
# load the bottleneck features from the timeseries model
timeseries_bottleneck = pd.read_csv('./bottleneck_features/bnf_timeseries.csv', parse_dates=True, index_col=0)

In [7]:
# load the bottleneck features from the renewable generation model
solar_bottleneck = pd.read_csv('./bottleneck_features/bnf_solar.csv', parse_dates=True, index_col=0)
wind_bottleneck = pd.read_csv('./bottleneck_features/bnf_wind.csv', parse_dates=True, index_col=0)

Historical generation, generation forecasts, loads and forecasted loads are available in quarter hour resolution. Since bottleneck features as well as Day-Ahead prices are given in an hourly resolution. Therefore, we resample the generation data to hourly intervals by taking the mean value over all quarter hour intervals of one individual hour (the unit of the resampled data is power, not energy. Thus averaging over all quarter hours is legitimate). 

In [8]:
# resample input data
actual = actual.resample('1H').mean()
forecast = forecast.resample('1H').mean()

All input datasets are merged into one big dataframe in order to align the indices and exclude missing data for all inputs simultaneously. This ensures consistent indexing among datasets.

In [9]:
# concatenate data 
features = pd.concat([actual, forecast, timeseries_bottleneck, solar_bottleneck, wind_bottleneck], axis=1)

# drop rows containing NaN values
features.dropna(inplace=True)

In [10]:
# ensure that features and labels have the same indexing by computing the index intersection of features and labels
index = features.index.intersection(price_data.index)

# pick features and labels with an index included in the intersection
features = features.loc[index]
labels = price_data.loc[index]

In [11]:
# double check feature and label indexing
labels.index.difference(features.index)

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

In [12]:
features.index.difference(labels.index)

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

In [13]:
# print names of the included datasets
features.columns

Index(['biomass', 'brown_coal', 'hard_coal', 'wind_offshore', 'pumped_hydro',
       'solar', 'river_hydro', 'wind_onshore', 'nuclear', 'other',
       'load_forecast', 'load_true', 'solar_forecast', 'offshore_forecast',
       'onshore_forecast', 'timeseries_pred', 'solar_bottleneck',
       'wind_bottleneck'],
      dtype='object')

In [14]:
features.head()

,biomass,brown_coal,hard_coal,wind_offshore,pumped_hydro,solar,river_hydro,wind_onshore,nuclear,other,load_forecast,load_true,solar_forecast,offshore_forecast,onshore_forecast,timeseries_pred,solar_bottleneck,wind_bottleneck
2015-01-01 01:00:00,4261.00,15364.75,1929.75,516.25,409.50,0.0,2617.00,8367.5,11086.25,4743.50,46952.50,47032.25,0.0,598.25,8161.75,16.366163,0.0,12577.362305
2015-01-01 02:00:00,4295.50,14852.75,1824.00,514.00,632.75,0.0,2578.75,8604.0,11026.25,4836.50,45751.50,45619.00,0.0,599.50,8324.75,13.697455,0.0,11986.717773
2015-01-01 03:00:00,4313.75,14111.00,1959.00,517.75,558.25,0.0,2545.25,8617.0,11027.75,4840.25,45306.25,44253.75,0.0,603.75,8440.25,10.958999,0.0,12911.827148
2015-01-01 04:00:00,4308.50,14149.00,2012.25,519.75,602.75,0.0,2557.75,8707.5,10962.25,4820.75,45423.00,43765.50,0.0,605.25,8621.25,10.259408,0.0,12743.263672
2015-01-01 05:00:00,4304.00,13509.50,1753.50,520.00,629.25,0.0,2554.75,8775.5,10696.00,4958.00,45701.50,43589.50,0.0,611.25,8825.75,11.003360,0.0,13013.442383


In [15]:
labels.head()

,DA_price
2015-01-01 01:00:00,18.29
2015-01-01 02:00:00,16.04
2015-01-01 03:00:00,14.60
2015-01-01 04:00:00,14.95
2015-01-01 05:00:00,14.50


# 2. Developing the final prediction model

Next, different regression models feeding on different subsets of the data are developed and evaluated. We will include data that is originally not available for our prediction in a real world setting in order to give a benchmark on the theoretical optimum of the modeling approach. This upper boundary serves as a reference for how far our model is from the 'perfect information' case.

In [16]:
def apply_predictor(predictor, features, labels, modelname='Some model'):
        
    """Perform a train-test split. Fit a predictor on the training set and evaluate error metric on the test
    set."""
    
    # perform randomized train-test split
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)
    
    # fit predictor and predict for test set
    predictor.fit(X_train, y_train)
    pred = predictor.predict(X_test)
    
    # compute performance metrics
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    med = median_absolute_error(y_test, pred)
    medape = np.median(np.abs((y_test - pred) / y_test) * 100)
    rmse = np.sqrt(mse)
    max_miss = max(abs(y_test - pred))
    r2 = r2_score(y_test, pred)
    
    print('{7:s} mean absolute error: {0:.2f}, median absolute error: {1:.2f}, mean squared error {2:.2f}, '
      'root mean squared error: {3:.2f}, median absolute percentage error: {4:.2f},  maximum deviation: {5:.2f},'\
      'r2 score: {6:.2f}'\
      .format(mae, med, mse, rmse, medape, max_miss, r2, modelname))
    
    return

## 2.1 Defining input feature sets 

**Idea:**

Different inputs might be fed into the regression model. The following input categories together with the hypotheses connected to them will be used and compared:
1. **'Perfect information':** Include all true major generation levels and the true load for the day of price prediction. Do not include any bottleneck features, especially not from the timeseries model. These inputs will serve as a reference point for the added value of the time series model.
2. **'Perfect information with time series bottleneck features':** By how much does including the precomputed time series based prediction improve overall model performance?
3. **'External renewable generation forecasts and true regular generation levels':** Are true renewable generation levels or renewable generation forecasts better inputs for price precition?
4. **'External renewable forecasts - added benefit of time series model':** Same as 2., true renewable generation levels are replaced by external renewable forcasts.
5. **'True renewable generation levels and timeseries bottleneck features':** Perfect information about renewable generation. Serves as benchmark for renewable bottleneck features.
6. **'External renewable generation forecasts and timeseries bottleneck features':** Second benchmark for renewable bottleneck features considering external forecasts instead of true renewable generation.
7. **'Final model':** All inputs that are available at before market closing, either external or precomputed by our own models are included. This is the 'real' prediction with information that would be available in a real world setting.
8. **'Renewable bottleneck without timeseries bottleneck:'** Benchmark for the benefit of the timeseries model in an actual deployment scenario.|

In [17]:
# define input features to be used in the different input subsets
cols1 = ['biomass', 'brown_coal', 'hard_coal', 'wind_offshore', 'pumped_hydro',
       'solar', 'river_hydro', 'wind_onshore', 'nuclear', 'other', 'load_true']

cols2 = ['biomass', 'brown_coal', 'hard_coal', 'wind_offshore', 'pumped_hydro',
       'solar', 'river_hydro', 'wind_onshore', 'nuclear', 'other', 'load_true',
       'timeseries_pred']

cols3 = ['biomass', 'brown_coal', 'hard_coal', 'pumped_hydro', 'river_hydro',
        'nuclear', 'other', 'load_forecast', 'solar_forecast', 'offshore_forecast',
        'onshore_forecast']

cols4 = ['biomass', 'brown_coal', 'hard_coal', 'pumped_hydro', 'river_hydro',
        'nuclear', 'other', 'load_forecast', 'solar_forecast', 'offshore_forecast',
        'onshore_forecast', 'timeseries_pred']

cols5 = ['wind_offshore', 'solar', 'wind_onshore', 'load_forecast', 'timeseries_pred']

cols6 = ['load_forecast', 'solar_forecast', 'offshore_forecast', 'onshore_forecast',
        'timeseries_pred']

cols7 = ['load_forecast', 'solar_bottleneck', 'wind_bottleneck', 'timeseries_pred']

cols8 = ['load_forecast', 'solar_bottleneck', 'wind_bottleneck']

input_variations = [cols1, cols2, cols3, cols4, cols5, cols6, cols7, cols8]

Now we can evaluate all different feature combinations and compare the results by applying a simple Linear Regression model.

In [19]:
for i, curr_feat in enumerate(input_variations):
    
    # evaluate current feature selection and print mean absolute error
    apply_predictor(LinearRegression(), features[curr_feat], labels.to_numpy().squeeze(), 'Feature set {}'.format(i+1))

Feature set 1 mean absolute error: 3.57, median absolute error: 2.79, mean squared error 22.95, root mean squared error: 4.79, median absolute percentage error: 8.88,  maximum deviation: 29.10,r2 score: 0.85
Feature set 2 mean absolute error: 3.38, median absolute error: 2.71, mean squared error 20.22, root mean squared error: 4.50, median absolute percentage error: 8.78,  maximum deviation: 25.29,r2 score: 0.87
Feature set 3 mean absolute error: 3.40, median absolute error: 2.61, mean squared error 21.27, root mean squared error: 4.61, median absolute percentage error: 8.30,  maximum deviation: 29.46,r2 score: 0.87
Feature set 4 mean absolute error: 3.28, median absolute error: 2.55, mean squared error 19.26, root mean squared error: 4.39, median absolute percentage error: 8.18,  maximum deviation: 24.39,r2 score: 0.88
Feature set 5 mean absolute error: 3.66, median absolute error: 2.88, mean squared error 23.79, root mean squared error: 4.88, median absolute percentage error: 9.14,  

Linear Regression is a very simple model. We can apply a more sophisticated algorithm in order to push the limits of predictive power a little further. Gradient Boosting generally shows a very good performance in all kinds of machine learing settings.

In [21]:
for i, curr_feat in enumerate(input_variations):
    
    # evaluate current feature selection and print mean absolute error
    apply_predictor(GradientBoostingRegressor(), features[curr_feat], labels.to_numpy().squeeze(), 'Feature set {}'.format(i+1))
    

Feature set 1 mean absolute error: 3.15, median absolute error: 2.31, mean squared error 17.99, root mean squared error: 4.24, median absolute percentage error: 7.93,  maximum deviation: 22.73,r2 score: 0.89
Feature set 2 mean absolute error: 3.05, median absolute error: 2.28, mean squared error 16.67, root mean squared error: 4.08, median absolute percentage error: 7.45,  maximum deviation: 21.85,r2 score: 0.89
Feature set 3 mean absolute error: 3.05, median absolute error: 2.22, mean squared error 16.98, root mean squared error: 4.12, median absolute percentage error: 7.41,  maximum deviation: 24.85,r2 score: 0.89
Feature set 4 mean absolute error: 2.94, median absolute error: 2.22, mean squared error 15.56, root mean squared error: 3.94, median absolute percentage error: 7.24,  maximum deviation: 20.96,r2 score: 0.90
Feature set 5 mean absolute error: 3.53, median absolute error: 2.62, mean squared error 22.55, root mean squared error: 4.75, median absolute percentage error: 8.30,  

**Analysis:**

The results contain some very interesting implications about the problem structure and the importance of some features compared to others. The main points are:

1. The best input features to our problem would be category 4, which are true generation, external renewable _forecasts_ and timeseries bottleneck features. These inputs yield better results than using the _true renewable generation_ inputs. This sounds counterintuitive in the first moment. Why are forecasted prediction levels better inputs than actual prediction levels? Thinking twice, the reason for that behavior is quite obvious. Day-Ahead prices are determined by _expectations_ of the market participants since prices are fixed before actual generation is known. The external forcasts are a representation of these expectations, aggregated by the TSOs after market clearing. Thus, they are a better description of the bidding strategies of all market participants than the actual generation levels. Hence, it would be better to predict _expected_ generation instead of _true_ generation as inputs for a price prediction model. This finding legitimates the usage of historical weather data for modeling actual generation levels, as we did with our renewable generation model. The model would perform even better if we fed it with weather forecasts instead of weather measurements. This assumes that the market participants base their individual expectations on similar weather forecasts than we use for the model. Forecasts from the German National Weather Service are an example for data with enough 'authority' to fulfill this assumption. Since our renewable generation model is able to predict real generation from real weather it seems fair to assume that it is able to predict expected generation based on expected weather conditions (forecasts). This hypothesis is supported by tests on category 5 and 6 (renewable forecasts vs renewable actual levels together with load forecasts and bottleneck features).
2. The comparison of results for category 1 vs 2 and 3 vs 4 (3 and 4 include the time series bottleneck features) show that our time series model features are helpful for the overall model precision.
3. The final model performance is actually not that far from perfect information. With a mean absolute error of 3.57€ per MWh is comparable to the benchmarks with external renewable forecasts (category 6, 3.45€ per MWh) and true renewable generation data (category 5, 3.53€ per MWh). The information that we would need to improve model performance substantially are estimations of non-renewable generation levels (e.g. brown coal, hard coal or nuclear). It might be possible to model these factors in additional models but this is out of the scope of this project.
4. Including the time series bottleneck features is quite important for the final model, as the comparison of category 7 and 8 shows. The effect is significantly stronger than in the scenarios where we have additional features available. The time series features most likely include some information that otherwise would come from the generation level features. Since we are not considering them explicitly, including the time series model is quite crucial for the end result. 

---


## 2.2 Optimizing model parameter inputs via grid search cross validation

In total, the performance of our final model is quite promising! We might be able to squeeze out a little more performance by tuning the hyperparameters of the gradient boosting algorithm. A grid search cross validation should help to find the optimal parametrization for our model.

> **Caution:** The grid search might take some time. Expect ~6 hours of runtime.

In [ ]:
# define grid search parameters
parameters = {'loss':('ls', 'lad', 'huber'), 'learning_rate':[0.05, 0.1, 0.3], 'n_estimators':[100, 1000, 5000],
              'min_samples_split':[2, 10, 20], 'max_depth':[3, 5, 10]}

# initialize predictor
predictor = GradientBoostingRegressor(random_state=7)

# initialize grid search instance
clf = GridSearchCV(predictor, parameters, verbose=2)

# iterate over parameter combinations to find the best parametrization
clf.fit(features[cols7].to_numpy(), labels.to_numpy().squeeze())

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV] learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=100 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=100, total=   0.9s
[CV] learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=100, total=   0.9s
[CV] learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=100 
[CV]  learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=100, total=   0.9s
[CV] learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=100 
[CV]  learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=100, total=   0.9s
[CV] learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=100 
[CV]  learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=100, total=   0.9s
[CV] learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=1000 
[CV]  learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=1000, total=   8.7s
[CV] learning_rate=0.05, loss=ls, max_depth=3, min_samples_split=2, n_estimators=1000 
[CV]  learning_rate=0.05, loss=ls, max_depth=3, min_samples_sp

In [ ]:
# print best parameter configuration and best score 
print(clf.best_params_, clf.best_score_)
best_params = clf.best_params

In [ ]:
# save best reference model
joblib.dump(clf, './models/final_model_gridsearch.pkl')
joblib.dump(clf.best_estimator_, './models/final_model_best_estimator.pkl');

In [ ]:
# print full estimator configuration
clf.best_estimator_

In [ ]:
best_params = {'learning_rate': 0.05, 'loss': 'huber', 'max_depth': 5, 'min_samples_split': 20, 'n_estimators': 100}

# 3. Analysis of final model performance

The best parameter combination of the grid search is depicted above. Since no explicit test set has been withdrawn from the data before grid searching, we have to rerun the model with the best parameter combination again on a train-test split of the data. Although the grid search applies a three fold cross validation, the final best model has been fit to all data passed to the grid search instance. As a result, the model might be overfitted to the data. To exclude the possibility of overfitting, we will retrain the model, this time keeping an explicit unseen test set for the final evaluation.

In [ ]:
# split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(features[cols7], labels, test_size=0.2, random_state=0)

In [ ]:
# mean absolute error of the fitted estimator from the grid search run
print(mean_absolute_error(y_test, clf.best_estimator_.predict(X_test)))

In [ ]:
# re-training of the model with the specified best parameter combination
apply_predictor(GradientBoostingRegressor(**best_params), features[cols7], labels.squeeze(), 'Final model')

**Analysis:**

As we can see above, the performance on **in-sample data** is significantly better than for **out-of-sample data**. Using the grid search estimator, we can achieve a mean absolute error of 3.11, while the performance on an unseen test set is around 3.46 mean absolute error.

The **final representative model performance** can be summarized as follows: The yearly average out of sample absolute error lies at 3.46€ per MWh. This refers to a root mean square error of 4.72€ per MWh and a median absolute percentage error of 8.39%. 

---

## 3.1 Reference benchmarks - Margin of improvement

Below, some reference points for the final model are given. First, the performance of a model with perfect forecasting information for renewable generation is given. This represents a benchmark for what we would be able to achieve with the _chosen modeling approach_ at best. Ideally, the model would be based on a weather forecast which is used to predict the renewable generation forecast, as explained above.

Second, a reference point for a model with total perfect information about all generation types is given. This benchmark represents a best case scenario in which we would have perfect information about all generation input parameters, togehter with the time series bottleneck features.

In [ ]:
best_params['n_estimators']=1000
apply_predictor(GradientBoostingRegressor(**best_params), features[cols6], labels.squeeze(), 'Benchmark model')

In [ ]:
best_params['n_estimators']=5000
apply_predictor(GradientBoostingRegressor(**best_params), features[cols4], labels.squeeze(), 'Perfect information model')

As we can see, there is quite a margin of improvement towards the first and second benchmark. Better renewable forecasts would be able to improve the overall model performance quite signigicantly. The second benchmark shows, that additional effort for generating other input features, like brown or hard coal generation prediction, has the potential to improve the model even further. This should be considered in possible future steps to improve the model.

---

## 3.2 Model evaluation on full-month out of sample data

Below, the final model performance is evaluated on a monthly test set. Instead of picking a random test set of the 2015 data, we use one complete month at a time as a test set. In recent literature about Day-Ahead price forecasting, model performance is often evaluated on a monthly or even weekly basis. Since there are 'easy' and 'hard' month in a year, this is only comparable to our results if we provide some 'best' and 'worst' case prediction performance metrics. 

In [ ]:
def apply_predictor_monthly(predictor, features, labels, month, modelname='Some model'):
        
    """Perform a train-test split with a full month as test data. Fit a predictor on
    the training set and evaluate error metric on the test
    set."""
    
    # pick indices for one month only
    index = features[features.index.month==month].index
    X_test, y_test = features.loc[index], labels.loc[index]
    X_train, y_train = features.drop(index), labels.drop(index)

    # fit predictor and predict on test set
    predictor.fit(X_train, y_train)
    pred = predictor.predict(X_test)
    
    # compute performance metrics
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    med = median_absolute_error(y_test, pred)
    medape = np.median(np.abs((y_test - pred) / y_test) * 100)
    rmse = np.sqrt(mse)
    max_miss = max(abs(y_test - pred))
    r2 = r2_score(y_test, pred)
    
    print('{7:s} mean absolute error: {0:.2f}, median absolute error: {1:.2f}, mean squared error {2:.2f}, '
      'root mean squared error: {3:.2f}, median absolute percentage error: {4:.2f},  maximum deviation: {5:.2f},'\
      'r2 score: {6:.2f}'\
      .format(mae, med, mse, rmse, medape, max_miss, r2, modelname))
    
    return

In [ ]:
# re-training of the model with the specified best parameter combination
best_params['n_estimators']=100
for m in range(1, 13, 1):
    print('Results for month {}'.format(m))
    apply_predictor_monthly(GradientBoostingRegressor(**best_params), features[cols7], labels.squeeze(), m, 'Final model')